## Content Based Recommender System

In [ ]:
import numpy as np 
import pandas as pd

: 

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head(2)

In [ ]:
movies.shape

In [ ]:
credits.head()

In [ ]:
credits.shape

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.head(2)

In [ ]:
movies.shape

In [ ]:
# Keeping important columns for recommendation
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head(2)

In [ ]:
movies.shape

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.shape

In [ ]:
movies.duplicated().sum()

In [ ]:
# handle genres

movies.iloc[0]['genres']

In [ ]:
import ast #for converting str to list

def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
# handle keywords
movies.iloc[0]['keywords']

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

In [ ]:
# handle cast
movies.iloc[0]['cast']

In [ ]:
# Here i am just keeping top 3 cast

def convert_cast(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert_cast)
movies.head()

In [ ]:
# handle crew

movies.iloc[0]['crew']

In [ ]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies.head()

In [ ]:
# handle overview (converting to list)

movies.iloc[0]['overview']

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.sample(4)

In [ ]:
movies.iloc[0]['overview']

In [ ]:
# now removing space like that 
'Anna Kendrick'
'AnnaKendrick'

def remove_space(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:

movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)

In [ ]:
movies.head()

In [ ]:
# Concatinate all
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
movies.iloc[0]['tags']

In [ ]:
# droping those extra columns
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df.head()

In [ ]:
# Converting list to str
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
new_df.head()

In [ ]:
new_df.iloc[0]['tags']

In [ ]:
# Converting to lower case
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
new_df.iloc[0]['tags']

In [ ]:
import nltk
from nltk.stem import PorterStemmer

In [ ]:
ps = PorterStemmer()

In [ ]:
def stems(text):
    T = []
    
    for i in text.split():
        T.append(ps.stem(i))
    
    return " ".join(T)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stems)

In [ ]:
new_df.iloc[0]['tags']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')


In [ ]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vector[0]

In [ ]:
vector.shape

In [ ]:
len(cv.get_feature_names())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity.shape

In [ ]:
# similarity

In [ ]:
new_df[new_df['title'] == 'The Lego Movie'].index[0]

In [ ]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Spider-Man 2')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('artifacts/movie_list.pkl','wb'))
pickle.dump(similarity,open('artifacts/similarity.pkl','wb'))